In [1]:
# Importação das bibliotecas a serem utilizadas
import requests
import pandas as pd

In [3]:
# Função que retorna uma lista de proposicoes, dados macro
def consulta_propostas():

    # Faz requisição para a API e obtém as informações das proposições em tramitação
    response = requests.get(url='https://dadosabertos.camara.leg.br/api/v2/proposicoes?ano=2022&siglaTipo=PL&itens=100')

    # Inicializa lista dos resultados
    ids_proposicoes = {}
    keep_loop = True

    while keep_loop:
    
        # Verifica se a requisição foi bem sucedida
        if response.status_code == 200:
            keep_loop = True
        else:
            print('Erro ao acessar a API')
            return []

        # Arquivo JSON retornado pela API
        proposicoes = response.json()['dados']

        # Loop na lista para extrair os IDs e URIs das proposições e retorna um dicionário
        for proposicao in proposicoes:
            ids_proposicoes[proposicao['id']] = proposicao['uri']

        #Verifica quantidade de paginas para consulta
        check_next = response.json()['links']

        for next in check_next:
            if next['rel'] == 'next':
                keep_loop = True
                next_request = next['href']
                break
            else:
                keep_loop = False

        df_detalhes = consulta_detalhes_propostas(ids_proposicoes)

        # Faz requisição para a API e obtém as informações das proposições em tramitação
        response = requests.get(url=next_request)

    return ids_proposicoes

In [2]:
# Função que retorna uma lista detalhaada das proposições passadas via parametro "ids_proposicoes"
def consulta_detalhes_propostas(ids_proposicoes):
    
    detalhes_proposicoes = []

    for id_proposicao in ids_proposicoes:

        response = requests.get(ids_proposicoes[id_proposicao])

        if response.status_code == 200:
            detalhes_proposicoes.append(response.json()['dados']) 
        else:
            print('Erro ao acessar a API')

    return detalhes_proposicoes

In [26]:
df = pd.DataFrame(consulta_detalhes_propostas(consulta_propostas()))

In [24]:
df = consulta_propostas()